## Ejercicios Web Scraping BeautifulSoup

In [1]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import re
from time import sleep

De la pagina:

https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html

- Sacar la siguiente información de las recetas de la primera página:

1. **Nombre**
2. **Descripcion**
3. **Comensales**
4. **Tiempo**
5. **Categoria**
6. **Dificultad**
7. **Ingredientes**
8. **Instrucciones**

Guardalo en un DataFrame y luego en un **`.csv`**.

AQUI NO VAMOS A USAR NI SELENIUM NI HELIUM:

1º guardamos la pagina web en una variable:

In [258]:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"

2º Guardamos la respuesta en una variable, si es 200 o 300 podremos extraer datos:


In [259]:
response = requests.get(url)
print(response)


<Response [200]>


3º Creamos una sopa:

In [260]:
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
soup.text

"\n\n\n\n\n\n808 Recetas de Ensaladas - ¡Fáciles, Frías, Originales y Variadas!  \n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDescubre\n\n\n\n\nAperitivos y tapas\nArroces y cereales\nAves y caza\nCarne\nCócteles y bebidas\nConsejos de cocina\nEnsaladas\nGuisos y Potajes\nHuevos y lácteos\nLegumbres\nMariscos\nPan y bollería\nPasta\nPescado\nPostres\nSalsas\nSopas y cremas\nVerduras\n\n\n \n\n\n\n\n\n\n\nBuscar\n\n\n\n\n\n\nRecetas\nEnsaladas\n\n\n\n Recetas de Ensaladas (808)\n\n\n¡Las mejores recetas de ensaladas! Aprende a preparar ensaladas fáciles y nutritivas, para bajar de peso o para disfrutar de un plato completo y refrescante. Recetas de ensaladas verdes, recetas de ensaladas de frutas e, incluso, recetas de ensaladas dulces es lo que te ofrecemos en este listado. Además, compartimos información interesante acerca del valor nutricional de las ensaladas, del número de calorías o de los posibles acompañamientos. Así, tanto si sigues una dieta para adelgaza

In [261]:
#viendo el cuerpo de mi sopa:
soup.body

<body class="ctrl-search action-index">
<div class="container">
<a name="top"></a>
<div class="cerrar-menu"></div>
<div class="menu" data-js-selector="menu">
<div class="wrap-menu max_width clearfix" data-js-selector="status">
<div class="descubre">
<div class="table">
<div class="centrar">Descubre</div>
</div>
</div>
<div class="lista_menu">
<div class="clear">
<a class="ga" data-category="Header" data-event="Go to hub" data-label="1" href="https://www.recetasgratis.net/Recetas-de-Aperitivos-tapas-listado_receta-1_1.html"><div class="hubicon hubicon--menu hubicon--aperitivos-y-tapas"></div>Aperitivos y tapas</a>
<a class="ga" data-category="Header" data-event="Go to hub" data-label="9" href="https://www.recetasgratis.net/Recetas-de-Arroces-cereales-listado_receta-9_1.html"><div class="hubicon hubicon--menu hubicon--arroces-y-cereales"></div>Arroces y cereales</a>
<a class="ga" data-category="Header" data-event="Go to hub" data-label="11" href="https://www.recetasgratis.net/Recetas-de-

In [8]:
soup.title.text

'808 Recetas de Ensaladas - ¡Fáciles, Frías, Originales y Variadas!'

4º Empezamos a filtrar la sopa por el titulo de la receta:

In [9]:
filtrado1 = soup.find_all("a", class_= "titulo titulo--resultado")


In [10]:
# Tenemos que convertir el filtrado en texto para poder seleccionar lo que queremos:
recetas = []
for enlace in filtrado1:
    texto = enlace.text
    #separamos el texto por ">" por ejemplo
    frases = texto.split(">")
    #se ha dividido en dos, y lo que nos interesa es el ultimo elemento del split
    recetas.append(frases[-1])


In [99]:
def obtener_recetas():
    filtrado1 = soup.find_all("a", class_= "titulo titulo--resultado")
    
    for enlace in filtrado1:
        texto = enlace.text
    #separamos el texto por ">" por ejemplo
        frases = texto.split(">")
    #se ha dividido en dos, y lo que nos interesa es el ultimo elemento del split
        recetas.append(frases[-1])
    return recetas

In [11]:
len(recetas)

50

5º Segundo filtrado por descripción:

In [12]:
filtrado2 = soup.find_all("div", class_= "intro")


In [13]:
# Tenemos que convertir el filtrado en texto para poder seleccionar lo que queremos:
descripcion = []
for enlace in filtrado2:
    #antes de convertir en lista, se puede usar el replace, despues no
    texto = enlace.text.replace("\n", "")
    #separar un texto por varias cosas: (necesario importar re)
    frases = re.split(r'<|>', texto)

    #se ha dividido en dos, y lo que nos interesa es el ultimo elemento del split
    descripcion.append(frases[0])

In [100]:
def obtener_descripcion():
    filtrado2 = soup.find_all("div", class_= "intro")
    
    for enlace in filtrado2:
    #antes de convertir en lista, se puede usar el replace, despues no
        texto = enlace.text.replace("\n", "")
    #separar un texto por varias cosas: (necesario importar re)
        frases = re.split(r'<|>', texto)

    #se ha dividido en dos, y lo que nos interesa es el ultimo elemento del split
        descripcion.append(frases[0])
    return descripcion

In [14]:
len(descripcion)

50

6º Filtrado 3 por comensales:

In [15]:
comensales = []
#como hay recetas que no tienen comensales, tenemos que entrar en el bloque de la receta:
for bloque in soup.find_all("div", class_= "resultado link"):
    try:
        if bloque.find("span", class_= "property comensales").text:
            comensales.append(bloque.find("span", class_= "property comensales").text)
    except:
        comensales.append(np.nan)

In [16]:
len(comensales)

50

In [101]:
def obtener_comensales():
    #como hay recetas que no tienen comensales, tenemos que entrar en el bloque de la receta:
    for bloque in soup.find_all("div", class_= "resultado link"):
        try:
            if bloque.find("span", class_= "property comensales").text:
                comensales.append(bloque.find("span", class_= "property comensales").text)
        except:
            comensales.append(np.nan)
    return comensales

In [51]:
comensales[31]

nan

7º Filtrado por tiempo:

In [148]:
filtrado4 = soup.find_all("span", class_= "property duracion")

In [149]:
#hay recetas que no tienen duracion:

tiempo = []
for bloque in soup.find_all("div", class_= "resultado link"):
    try:
        if bloque.find("span", class_= "property duracion").text:
            tiempo.append(bloque.find("span", class_= "property duracion").text)
    except:
        tiempo.append(np.nan)

In [19]:
len(tiempo)

50

In [150]:
def obtener_tiempo():
    for bloque in soup.find_all("div", class_= "resultado link"):
        try:
            if bloque.find("span", class_= "property duracion").text:
                tiempo.append(bloque.find("span", class_= "property duracion").text)
        except:
            tiempo.append(np.nan)
    return tiempo

8º Filtrado por dificultad:

In [95]:
filtrado5 = soup.find_all("div", class_= "info_snippet")

In [96]:
dificultad = []
for enlace in filtrado5:
    texto = enlace.text.replace("\n", "")
    dificultad.append(texto)

In [98]:
dificultad

['Dificultad bajaNuevo ',
 'Dificultad media',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad muy bajaEspecial Navidad',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 '',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad baja',
 '',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad baja',
 'Dificultad muy baja',
 'Dificultad mu

In [97]:
dificultad[15]

'Dificultad muy bajaEspecial Navidad'

In [22]:
dificultad[15] = "Dificultad muy baja"

In [23]:
dificultad[0] = "Dificultad baja"

In [103]:
def obtener_dificultad():
    filtrado5 = soup.find_all("div", class_= "info_snippet")

    for enlace in filtrado5:
        texto = enlace.text.replace("\n", "")
        dificultad.append(texto)
    return dificultad

In [24]:
len(dificultad)

50

9º Filtrado por categoría:

In [151]:
categoria = []
for bloque in soup.find_all("div", class_= "resultado link"):
    try:
        if bloque.find("span", class_= "property para").text:
            categoria.append(bloque.find("span", class_= "property para").text)
    except:
        categoria.append(np.nan)

In [153]:
len(categoria)

50

In [154]:
def obtener_categoria():
    for bloque in soup.find_all("div", class_= "resultado link"):
        try:
            if bloque.find("span", class_= "property para").text:
                categoria.append(bloque.find("span", class_= "property para").text)
        except:
            categoria.append(np.nan)
    return categoria

10º Filtrado por ingredientes:

In [28]:
# Encuentra todas las etiquetas <a> con la clase "titulo titulo--resultado"
filtrado7 = soup.find_all("a", class_="titulo titulo--resultado")

In [29]:
enlaces = [enlace["href"] for enlace in filtrado7]

In [171]:
lista_ingredientes = []
for enlace in enlaces:
    #vemos si podemos acceder al enlace
    #Guardamos la respuesta en una variable, si es 200 o 300 podremos extraer datos:
    response = requests.get(enlace)
    #status code para acceder al numero de la respuesta
    if response.status_code == 200:
        # Crea un objeto BeautifulSoup para analizar el contenido de la página
        soup3 = BeautifulSoup(response.content, "html.parser")
        lista = soup3.find_all("li", class_="ingrediente")
        len(lista)
        ingredientes = [x.get_text(strip=True) for x in lista]
                
        lista_ingredientes.append(ingredientes)
         

In [105]:
def obtener_ingredientes():
    filtrado7 = soup.find_all("a", class_="titulo titulo--resultado")
    enlaces = [enlace["href"] for enlace in filtrado7]
    for enlace in enlaces:
    #vemos si podemos acceder al enlace
    #Guardamos la respuesta en una variable, si es 200 o 300 podremos extraer datos:
        response = requests.get(enlace)
    #status code para acceder al numero de la respuesta
        if response.status_code == 200:
        # Crea un objeto BeautifulSoup para analizar el contenido de la página
            soup3 = BeautifulSoup(response.content, "html.parser")
            lista = soup3.find_all("li", class_="ingrediente")
        
            ingredientes = [x.get_text(strip=True) for x in lista]
                
            lista_ingredientes.append(ingredientes)

    return lista_ingredientes

In [31]:
len(lista_ingredientes)

50

In [42]:
#Voy a ver en qué indices por ejemplo de la lista tiempo, están los nans
indices_nan = [i for i, x in enumerate(tiempo) if isinstance(x, float) and np.isnan(x)]

In [43]:
indices_nan

[20, 31]

In [47]:
#voy añadir un elemento nan en esos indices:
for i in indices_nan:
    lista_ingredientes[i] = np.nan
    print(lista_ingredientes[i])

nan
nan


In [50]:
#tambien se los voy a añadir a dificultad:
for i in indices_nan:
    dificultad[i] = np.nan
    print(dificultad[i])

nan
nan


11º Filtrado para instrucciones:

- Repetir el ejercicio para todas las recetas de todas las páginas.

In [340]:

instrucciones = []

def obtener_instrucciones():
    lista_instrucciones = []
    for apartado in soup.find_all("a", class_="titulo titulo--resultado"):
        
        enlace = apartado["href"]
        respuesta = requests.get(enlace)

        if respuesta.status_code == 200:
            sopa_apartado = BeautifulSoup(respuesta.content, "html.parser")
            #tengo la sopa de cada apartado de la pagina en la que estoy
            for orden in range(len(sopa_apartado.find_all("div", class_="orden"))):
                #itera el numero de ordenes que tiene cada sopa_apartado
                anchor = sopa_apartado.find_all("div", class_ = "apartado", attrs={"id":f"anchor_{orden}"})
                #entramos en la clase que nos interesa, donde están las instrucciones
                #convertimos en texto:
                for texto in anchor:
                    elemento = texto.text.replace(f"\n{orden+1}", "")
                    lista_instrucciones.append(elemento.replace("\n", ""))
        instrucciones.append(lista_instrucciones)
        lista_instrucciones = []
        
         


FILTRADO POR PAGINA:

1. **Nombre**
2. **Descripcion**
3. **Comensales**
4. **Tiempo**
5. **Categoria**
6. **Dificultad**
7. **Ingredientes**
8. **Instrucciones**

In [351]:
#Voy a definir una funcion que obtenga los enlaces de una página:
url = "https://www.recetasgratis.net/Recetas-de-Ensaladas-listado_receta-4_1.html"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
recetas = []
descripcion = []
comensales = []
tiempo = []
categoria = []
dificultad = []
lista_ingredientes = []
instrucciones = []

obtener_recetas()
obtener_descripcion()
obtener_comensales()
obtener_dificultad()
obtener_tiempo()
obtener_categoria()
obtener_ingredientes()
obtener_instrucciones()

In [355]:
len(instrucciones)

808

In [354]:
while True:
    try:
        enlace = soup.find("a", class_="next ga").get("href")
    
        if enlace:
        
            siguiente_pagina = enlace
            response2 = requests.get(siguiente_pagina)
            soup = BeautifulSoup(response2.content, "html.parser")

            obtener_recetas()
            obtener_descripcion()
            obtener_comensales()
            obtener_dificultad()
            obtener_tiempo()
            obtener_categoria()
            obtener_ingredientes()
            obtener_instrucciones()
            

        
        if not enlace:  # Si no hay enlaces de siguiente página, sal del bucle
            break
    except:        
        break

SyntaxError: incomplete input (422270377.py, line 24)

In [365]:
#crear df:
data = {
    'Recetas': recetas,
    'Descripcion': descripcion,
    'Comensales': comensales,
    'Tiempo': tiempo,
    'Categoria': categoria,
    'Dificultad': dificultad,
    'Ingredientes': lista_ingredientes,
    'Instrucciones': instrucciones
}

# Crear el DataFrame
df = pd.DataFrame(data)

# Imprimir el DataFrame
print(df)

                                               Recetas  \
0                 Receta de Ensalada de pasta con atún   
1                         Receta de Ensaladilla vegana   
2     Receta de Ensalada de garbanzos veraniega rápida   
3                              Receta de Moje murciano   
4         Receta de Ensalada de garbanzos mediterránea   
..                                                 ...   
803                       Receta de Ensalada de granos   
804                        Receta de Ensalada de papas   
805            Receta de Ensalada de coditos con queso   
806  Receta de Ensalada de zanahorias con piña y pasas   
807                          Receta de Ensalada azteca   

                                           Descripcion Comensales Tiempo  \
0    La ensalada de pasta con atún y huevo es una r...          5    30m   
1    La ensaladilla vegana consiste en un plato tra...          4    45m   
2    En verano con el calor apetecen mucho las ensa...          4    15m   

In [366]:
# Guardar en un archivo CSV
df.to_csv('recetas_csv.csv', index=False)  # index=False evita que se guarde el índice del DataFrame
# Guardar en un archivo Excel (xlsx)
df.to_excel('recetas_excel.xlsx', index=False)  # index=False evita que se guarde el índice del DataFrame

In [356]:
len(instrucciones)

808

In [357]:
print(len(recetas))


808


In [358]:
print(len(descripcion))


808


In [359]:
print(len(comensales))


808


In [360]:
print(len(tiempo))

808


In [361]:

print(len(categoria))

808


In [362]:
print(len(dificultad))

808


In [363]:

print(len(lista_ingredientes))


808


In [364]:
len(instrucciones)

808

In [1]:
type(tiempo[20])

NameError: name 'tiempo' is not defined